In [10]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None   # default='warn'

data = pd.read_excel('cleaned_data.xlsx')

In [11]:
data['Price_per_unit_area'] = data['Price_per_unit_area'].astype(str).str.replace(',', '').astype(int)
data['Price'] = data['Price'].astype(str).str.replace(',', '').astype(int)
data['Size'].astype(str)
data['Size'] = data['Size'].str.removesuffix(' sq ft').str.replace(',', '').astype(int)

In [12]:
attrbs = ['Property_Name', 'Property_id', 'Project_URL', 'builder_id', 'Builder_name', 'Locality_ID', 'Longitude', 'Latitude', 'Sub_urban_ID', 'description', 'City_id', 'Listing_Category', 'is_plot', 'is_RERA_registered', 'is_Apartment', 'is_ready_to_move', 'is_commercial_Listing', 'is_PentaHouse', 'is_studio', 'listing_domain_score', 'No_of_BHK', 'Locality_Name', 'Sub_urban_name', 'Posted_On', 'Property_building_status', 'is_furnished']

for att in attrbs:
    data.drop(att, axis=1, inplace=True)

In [13]:
def priceG(dataset):
    _gold = ((data['Price'] <= 8000000000) & (data['Price'] >= 50000000))
    _silver = ((data['Price'] < 50000000) & (data['Price'] >= 2000000))
    _bronze = ((data['Price'] < 2000000) & (data['Price'] >= 100000))

    dataset.loc[_bronze, ['Price']] = 'Bronze'
    dataset.loc[_silver, ['Price']] = 'Silver'
    dataset.loc[_gold, ['Price']] = 'Gold'

In [14]:
def sizeG(dataset):
    _large = ((data['Size'] <= 90000) & (data['Size'] >= 50000))
    _medium = ((data['Size'] < 50000) & (data['Size'] >= 5000))
    _small = ((data['Size'] < 5000) & (data['Size'] >= 100))

    dataset.loc[_large, ['Size']] = 'Large'
    dataset.loc[_medium, ['Size']] = 'Medium'
    dataset.loc[_small, ['Size']] = 'Small'

In [15]:
def work(a):
    priceG(a)
    sizeG(a)

In [16]:
data1 = data[(data['Property_status'] == 'Ready to move') & ((data['City_name'] == 'Hyderabad') | (data['City_name'] == 'Ahmedabad') | (data['City_name'] == 'Bangalore') | (data['City_name'] == 'Mumbai'))]
data1.drop(['Property_status', 'City_name', 'Price_per_unit_area'], axis=1, inplace=True)

In [17]:
work(data1)

In [18]:
data1 = data1.drop_duplicates()
data1

,Property_type,Price,Size
2,Apartment,Silver,Small
56,Villa,Silver,Small
64,Apartment,Bronze,Small
153,Independent House,Silver,Small
164,Independent Floor,Silver,Small
210,Villa,Silver,Medium
230,Residential Plot,Bronze,Small
232,Residential Plot,Silver,Small
235,Residential Plot,Silver,Medium
331,Apartment,Silver,Medium


In [19]:
data2 = data[(data['Property_status'] == 'Under Construction') & ((data['City_name'] == 'Chennai') | (data['City_name'] == 'Kolkata') | (data['City_name'] == 'Chennai') | (data['City_name'] == 'Lucknow')) & (data['Price_per_unit_area'] > 10000) & (data['Price_per_unit_area'] < 50000)]
data2.drop(['Property_status', 'City_name', 'Price_per_unit_area'], axis=1, inplace=True)

In [20]:
work(data2)

In [21]:
data2 = data2.drop_duplicates()
data2

,Property_type,Price,Size
27676,Apartment,Silver,Small
28529,Independent House,Silver,Small
29762,Apartment,Gold,Medium
30278,Apartment,Gold,Small
30968,Villa,Silver,Small
67415,Independent House,Gold,Small


Consider the 1st row. The rule states that *Small sized Apartment are priced in the Silver*. This can be represented as a logic formula as follows:

$\exists \; t \; ((\text{Size}(t) = \text{Small}) \wedge (\text{Property Type}(t) = \text{Apartment}) \wedge ((\text{Price}(t) = \text{Silver}) \vee (\text{Price}(t) = \text{Gold})) \wedge (\text{City}(t) = \text{Other City})$

Similiarly, other logic formulas can be extracted.